In [10]:
import pickle
import matplotlib.pyplot as plt

with open("/home/max/ET-TI/Masterarbeit/datasets/beatles/chroma/1_deep_chroma.pkl", "rb") as f:
    df = pickle.load(f)

# Group the DataFrame by label
grouped = df.groupby('label')

for label, group in grouped:
    if label == "C#":
        x = group.drop('label', axis=1)

x

,C,C#,D,D#,E,F,F#,G,G#,A,A#,B
6171,0.19597152,0.0,0.0,0.0,0.21770333,0.0,0.0,0.5863252,0.0,0.0,0.0,0.0
6172,0.29632914,0.0,0.0,0.0,0.3128903,0.0,0.0,0.39078054,0.0,0.0,0.0,0.0
6173,0.29119706,0.0,0.0,0.0,0.33956844,0.0,0.0,0.3692345,0.0,0.0,0.0,0.0
6174,0.28970787,0.0,0.0,0.0,0.30690622,0.0,0.0,0.40338597,0.0,0.0,0.0,0.0
6175,0.24966185,0.0,0.0,0.0,0.31247395,0.0,0.0,0.4378642,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
29380,0.0,0.52761775,0.0,0.0,0.0,0.0918643,0.0,0.0,0.380518,0.0,0.0,0.0
29381,0.0,0.5380955,0.0,0.0,0.0,0.0604706,0.0,0.0,0.4014339,0.0,0.0,0.0
29382,0.0,0.5934081,0.0,0.0,0.0,0.0,0.0,0.0,0.4065919,0.0,0.0,0.0
29383,0.0,0.66784054,0.0,0.0,0.0,0.0,0.0,0.0,0.3321595,0.0,0.0,0.0
